In [1]:
from impala.dbapi import connect
conn = connect(host='ip-10-0-0-84.ap-northeast-1.compute.internal', port=21050)

In [2]:
cur = conn.cursor()

cur.execute('select * from default.sentiment_data limit 2')

from impala.util import as_pandas
df = as_pandas(cur)

df.head()

,sentiment,body
0,0,It’s ‘Star Trek’ vs. ‘Star Wars’ in European S...
1,0,$JD $AAPL


In [7]:
cur = conn.cursor()

cur.execute('select * from default.sentiment_data limit 2')

list = cur.fetchall()
list

[(0,
  'It’s ‘Star Trek’ vs. ‘Star Wars’ in European Streaming Battle  $CMCSA $VIAC $AAPL $AMZN $DIS '),
 (0, '$JD $AAPL')]

In [4]:
import csv

In [8]:
with open('./sample_output.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerows(list)

In [9]:
!cat sample_output.csv

In [1]:
%load_ext sql

In [2]:
%%sql
impala://ip-10-0-0-84.ap-northeast-1.compute.internal:21050/default

'Connected: None@default'

In [3]:
%%sql
select * from default.sentiment_data limit 2

Done.


sentiment,body
0,It’s ‘Star Trek’ vs. ‘Star Wars’ in European Streaming Battle $CMCSA $VIAC $AAPL $AMZN $DIS
0,$JD $AAPL


In [ ]:
%%sql
select * from default.sentiment_data limit 2